In [5]:
import os
import sys
sys.path.append('../va_pipeline/')

In [1]:
import subprocess
import pandas as pd
import cv2
import numpy as np

from sensor import *
clear_chart()

In [2]:
from win32gui import GetWindowText, GetForegroundWindow
import time
time.sleep(5)
print(GetWindowText(GetForegroundWindow()))

UM25C PC Software V1.3
